In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime as dt

from scipy.spatial import Delaunay
from scipy.stats import linregress
from scipy.optimize import curve_fit
from getdist import plots, MCSamples
from iminuit import Minuit
from iminuit.cost import LeastSquares

from qubic.lib.Qgps import GPSAntenna
import qubic.lib.Calibration.Qfiber as ft
 
%matplotlib inline

In [ ]:
# If True, allow plots for debug
DEBUG = False

# Import data

In [ ]:
### Build GPS data file path
data_path = "calsource_orientation.dat"

### Define the distance between the two antennas
distance_between_antennas = 1.3 # Just to initialize GPSAntenna

In [ ]:
gps_antenna = GPSAntenna(data_path, distance_between_antennas)

names = np.array(['North', 'East', 'Down', 'Roll', 'Yaw'])

del data_path, distance_between_antennas

In [ ]:
index_ini = gps_antenna.get_observation_indices(gps_antenna._datetime, np.array([dt.datetime(year=2024, month=12, day=12, hour=8, minute=55, second=0)]))[0]
print('Initial index = ', index_ini)
print('size observation times : ', len(gps_antenna._datetime))

In [ ]:
gps_antenna.plot_gps_data(index_start=index_ini)

# Build observation time indices

In [ ]:
# Array containing the starting time of each configuration
array_hours = np.array([8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11])
array_minutes_begins = np.array([55, 0, 6, 12, 19, 25, 32, 41, 46, 52, 58, 4, 10, 19, 27, 34, 40, 46, 53, 1, 7, 13, 20, 25])

# Array containing the ending time of each configuration, build by substracting 1 minute to the starting time
# In this case, each time correspond to do the ending time of the previous configuration
array_minutes_ending = array_minutes_begins.copy() - 1
array_minutes_ending[1] = 0

# Array containing the seconds of each configuration
array_seconds_begins = np.ones(array_hours.shape, dtype=int) * 59
array_seconds_ending = np.zeros(array_hours.shape, dtype=int)

# Build the array of datetime associated with each configuration, to delimit the time when the antenna are moving or not
array_datetime = np.array([])
index_array = np.array([], dtype=int)
for index in range(array_hours.shape[0]):
    array_datetime = np.append(array_datetime, dt.datetime(year=2024, month=12, day=12, hour=array_hours[index], minute=array_minutes_begins[index], second=array_seconds_begins[index]))
for index in range(1, array_hours.shape[0]):
    array_datetime = np.append(array_datetime, dt.datetime(year=2024, month=12, day=12, hour=array_hours[index], minute=array_minutes_ending[index], second=array_seconds_ending[index]))
array_datetime = np.append(array_datetime, dt.datetime(year=2024, month=12, day=12, hour=11, minute=33, second=0))

for index in range(array_datetime.shape[0]):
    index_array = np.append(index_array, gps_antenna.get_observation_indices(gps_antenna._datetime, np.array([array_datetime[index]])))
        
del array_hours, array_minutes_begins, array_minutes_ending, array_seconds_begins, array_seconds_ending, array_datetime, index

## Test

In [ ]:
if DEBUG:
    for index in range(int(index_array.size/2)):
        plt.figure(figsize=(15, 8))
        plt.plot(gps_antenna._datetime[index_ini:], gps_antenna.roll[index_ini:])
        plt.xlabel('Date')
        plt.ylabel('Angle (rad)')
        plt.title('GPS Roll Angle')
        plt.vlines(gps_antenna._datetime[index_array[index]], 0, 2*np.pi, color='r', linestyle='--')
        plt.vlines(gps_antenna._datetime[index_array[index + int(index_array.size/2)]], 0, 2*np.pi, color='r', linestyle='--')
        plt.show()
        
    del index

In [ ]:
# Plot to verify the previous delimitation
fig, ax1 = plt.subplots(figsize = (15,5))

color_a = 'tab:pink'
color_r = 'tab:red'
color_b = 'tab:blue'
color_d = 'tab:green'
color_c = 'tab:brown'

ax1.set_xlabel('Date')
ax1.set_ylabel('Position Vector Components (m)', color = color_r)
ax1.plot(gps_antenna._datetime[index_ini:], gps_antenna.rpN[index_ini:], color = color_r, label = 'North component')
ax1.plot(gps_antenna._datetime[index_ini:], gps_antenna.rpE[index_ini:], color = color_b, label = 'East component')
ax1.plot(gps_antenna._datetime[index_ini:], gps_antenna.rpD[index_ini:], color = color_d, label = 'Up component')

ax2 = ax1.twinx()

ax2.plot(gps_antenna._datetime[index_ini:], gps_antenna.roll[index_ini:], color = color_a, label = 'Roll angle')
ax2.plot(gps_antenna._datetime[index_ini:], gps_antenna.yaw[index_ini:], color = color_c, label = 'Yaw angle')
ax2.set_xlabel('Date')
ax2.set_ylabel('Angles (rad)', color = color_a)

for obs_index in index_array:
    ax1.vlines(gps_antenna._datetime[obs_index], 3.5, -1.5, 'grey', linestyles='--')
ax1.vlines(gps_antenna._datetime[index_array[0]], 3.5, -1.5, 'r', linestyles='--')
ax1.vlines(gps_antenna._datetime[index_array[0 + int(index_array.shape[0]/2)]], 3.5, -1.5, 'r', linestyles='--')

fig.tight_layout()
ax1.set_title("Position vector components")
fig.legend(bbox_to_anchor=(1, 1), loc='upper left')
plt.show()

del fig, ax1, ax2, color_a, color_r, color_b, color_d, color_c, obs_index

In [ ]:
# Function to remove the movement between each configurations
def only_data(array, indices):
    data_array = np.array([])
    
    for i in range(int(indices.shape[0]/2)):
        data_array = np.append(data_array, array[indices[i]:indices[i+int(indices.shape[0]/2)]])
    
    return data_array

In [ ]:
data_rpN = only_data(gps_antenna.rpN, index_array)
data_rpE = only_data(gps_antenna.rpE, index_array)
data_rpD = only_data(gps_antenna.rpD, index_array)
data_roll = only_data(gps_antenna.roll, index_array)
data_yaw = only_data(gps_antenna.yaw, index_array)

In [ ]:
data_rpN.shape

In [ ]:
index_array.shape

In [ ]:
# Plot to verify the filtering of the time when the antennas are moving
fig, ax1 = plt.subplots(figsize = (15,5))

color_a = 'tab:pink'
color_r = 'tab:red'
color_b = 'tab:blue'
color_d = 'tab:green'
color_c = 'tab:brown'

ax1.set_xlabel('Date')
ax1.set_ylabel('Position Vector Components (m)', color = color_r)
ax1.plot(data_rpN, color = color_r, label = 'North component')
ax1.plot(data_rpE, color = color_b, label = 'East component')
ax1.plot(data_rpD, color = color_d, label = 'Up component')

ax2 = ax1.twinx()

ax2.plot(data_roll, color = color_a, label = 'Roll angle')
ax2.plot(data_yaw, color = color_c, label = 'Yaw angle')
ax2.set_xlabel('Date')
ax2.set_ylabel('Angles (rad)', color = color_a)

fig.tight_layout()
ax1.set_title("Position vector components")
fig.legend(bbox_to_anchor=(1, 1), loc='upper left')
plt.show()

del fig, ax1, ax2, color_a, color_r, color_b, color_d, color_c

# Standard deviation for each configurations

In [ ]:
# Function to compute the standard deviation avoiding moments when I changed the experimental configuration
def only_data_std(array, indices):
    data_array = np.array([])
    
    for i in range(int(indices.shape[0]/2)):
        data_array = np.append(data_array, np.std(array[indices[i]:indices[i+int(indices.shape[0]/2)]]))
    
    return data_array

In [ ]:
# Multiply by 100 to convert from m to cm
std_rpN = only_data_std(gps_antenna.rpN, index_array) * 100
std_rpE = only_data_std(gps_antenna.rpE, index_array) * 100
std_rpD = only_data_std(gps_antenna.rpD, index_array) * 100
std_roll = np.degrees(only_data_std(gps_antenna.roll, index_array))
std_yaw = np.degrees(only_data_std(gps_antenna.yaw, index_array))

In [ ]:
# Distances during test 1
distance_base_antenna1_test1 = np.array([20, 20, 20, 20, 20, 100, 100, 100, 100, 200, 200, 200, 200])
distance_base_antenna2_test1 = np.array([20, 50, 100, 200, 300, 300, 200, 100, 50, 50, 100, 200, 300])

# Distance during test 2
distance_base_antenna2_test2 = np.array([300, 300, 300, 200, 200, 200, 200, 100, 100, 100, 100])
distance_antenna1_antenna2_test2 = np.array([50, 100, 200, 50, 100, 200, 300, 50, 100, 200, 300])

In [ ]:
# Plot the standard deviation for each instumental configuration
fig, ax1 = plt.subplots(figsize = (15,5))

color_a = 'tab:pink'
color_r = 'tab:red'
color_b = 'tab:blue'
color_d = 'tab:green'
color_c = 'tab:brown'

ax1.set_xlabel('Configurations index')
ax1.set_ylabel('Standard deviation of Position (cm)', color = color_r)
ax1.plot(std_rpN, label = 'rpN', color=color_r)
ax1.plot(std_rpE, label = 'rpE', color=color_b)
ax1.plot(std_rpD, label = 'rpD', color=color_d)
ax2 = ax1.twinx()

ax2.plot(std_roll, label = 'roll', color=color_a)
ax2.plot(std_yaw, label = 'yaw', color=color_c)
ax2.set_xlabel('Configurations index')
ax2.set_ylabel('Standard Deviation of Angles (degrees)', color = color_a)

fig.tight_layout()
ax1.set_title("Standard deviation of the GPS data for all experimental configurations")
fig.legend(bbox_to_anchor=(1, 1), loc='upper left')
plt.show()

del fig, ax1, ax2, color_a, color_r, color_b, color_d, color_c

In [ ]:
print('Minimum Std Position : ', np.min([std_rpD, std_rpE, std_rpN]).round(2), '(cm), Maximum Std Position : ', np.max([std_rpD, std_rpE, std_rpN]).round(2), '(cm)')
print('Minimum Std Angle : ', np.min([std_roll, std_yaw]).round(2), '(degrees), Maximum Std Angle : ', np.max([std_roll, std_yaw]).round(2), '(degrees)')

## Standard Deviation for Experiment 1

In [ ]:
def plot_std_exp1(array, pos=True):
    plt.figure()
    if pos:
        leg = 'cm'
    else:
        leg = 'deg'
    scatter = plt.scatter(distance_base_antenna2_test1, distance_base_antenna1_test1, s=array[:distance_base_antenna1_test1.shape[0]]*100, c=array[:distance_base_antenna1_test1.shape[0]], cmap='viridis')
    plt.colorbar(scatter, label=f'Standard deviation ({leg})')
    plt.xlabel('Distance Base-Antenna 2 (cm)')
    plt.ylabel('Distance Base-Antenna 1 (cm)')

In [ ]:
plot_std_exp1(std_rpN)
plt.title('Standard deviation of the North component of antenna 1')
plt.show()
plot_std_exp1(std_rpE)
plt.title('Standard deviation of the East component of antenna 1')
plt.show()
plot_std_exp1(std_rpD)
plt.title('Standard deviation of the Up component of antenna 1')
plt.show()
plot_std_exp1(std_roll, pos=False)
plt.title('Standard deviation of the roll angle of antenna 1')
plt.show()
plot_std_exp1(std_yaw, pos=False)
plt.title('Standard deviation of the yaw angle of antenna 1')
plt.show()

## Standard Deviation for Experiment 2

In [ ]:
def plot_std_exp2(array, pos=True):
    if pos:
        leg = 'cm'
    else:
        leg = 'deg'
    scatter = plt.scatter(distance_base_antenna2_test2, distance_antenna1_antenna2_test2, s=array[distance_base_antenna2_test1.shape[0]:]*100, c=array[distance_base_antenna2_test1.shape[0]:], cmap='viridis')
    plt.colorbar(scatter, label=f'Standard deviation ({leg})')
    plt.legend()
    plt.xlabel('Distance Base-Antenna 2 (cm)')
    plt.ylabel('Distance Antenna 1 - Antenna 2')

In [ ]:
plot_std_exp2(std_rpN)
plt.title('Standard deviation of the North component of antenna 1')
plt.show()
plot_std_exp2(std_rpE)
plt.title('Standard deviation of the East component of antenna 1')
plt.show()
plot_std_exp2(std_rpD)
plt.title('Standard deviation of the Up component of antenna 1')
plt.show()
plot_std_exp2(std_roll, pos=False)
plt.title('Standard deviation of the roll angle of antenna 1')
plt.show()
plot_std_exp2(std_yaw, pos=False)
plt.title('Standard deviation of the yaw angle of antenna 1')
plt.show()

# Incertainty on antenna 1 position

$\Delta r = \sqrt{(\frac{x}{r}\Delta x)^{2} + (\frac{y}{r}\Delta y)^{2} + (\frac{z}{r}\Delta z)^{2}}$

In [ ]:
delta_r = np.array([])

for i in range(int(index_array.shape[0]/2)):
    data_array_north = np.mean(gps_antenna.rpN[index_array[i]:index_array[i+int(index_array.shape[0]/2)]] )
    data_array_east = np.mean(gps_antenna.rpE[index_array[i]:index_array[i+int(index_array.shape[0]/2)]] )
    data_array_down = np.mean(gps_antenna.rpD[index_array[i]:index_array[i+int(index_array.shape[0]/2)]])

    # Don't forget to convert from m to cm
    std_north = np.std(gps_antenna.rpN[index_array[i]:index_array[i+int(index_array.shape[0]/2)]]) * 100
    std_east = np.std(gps_antenna.rpE[index_array[i]:index_array[i+int(index_array.shape[0]/2)]]) * 100
    std_down = np.std(gps_antenna.rpD[index_array[i]:index_array[i+int(index_array.shape[0]/2)]]) * 100

    r = np.sqrt(data_array_north**2 + data_array_east**2 + data_array_down**2)

    delta_r = np.append(delta_r, np.sqrt((data_array_north * std_north / r)**2 + 
                                         (data_array_east * std_east / r)**2 + 
                                         (data_array_down * std_down / r)**2))
print(delta_r.shape)

del i, data_array_north, data_array_east, data_array_down, std_north, std_east, std_down, r

In [ ]:
plt.figure()
scatter = plt.scatter(distance_base_antenna2_test1, distance_base_antenna1_test1, s=delta_r[:distance_base_antenna1_test1.shape[0]]*100, c=delta_r[:distance_base_antenna1_test1.shape[0]], cmap='viridis')
plt.colorbar(scatter, label='Standard deviation (cm)')
plt.xlabel('Distance Base-Antenna 2 (cm)')
plt.ylabel('Distance Base-Antenna 1 (cm)')
plt.title('Relative error on the position of the antenna 1 during experiment 1')

# Define function to make plot using Delaunay triangulation

In [ ]:
def delaunay_surface_plot(x, y, z, title, xtitle, ytitle, ztitle, save=False):
    tri = Delaunay(np.array([x, y]).T)
    simplices = tri.simplices

    fig = ff.create_trisurf(x=x, y=y, z=z,
                                simplices=simplices, title=dict(text=title, font=dict(size=24)),
                                height=1000, width=1000)
    
    fig.update_layout(scene = dict(xaxis_title=xtitle,
                                yaxis_title=ytitle,
                                zaxis_title=ztitle),
                                )
        
    if save:
        fig.write_html('plotly/plotly.html')
    fig.show()

# Experiment 1

## Position Antenna 1

In [ ]:
# Experiment 1
delaunay_surface_plot(distance_base_antenna1_test1, distance_base_antenna2_test1, delta_r[:distance_base_antenna1_test1.shape[0]], 
                      'Relative error on the position of the antenna 1 during experiment 1',
                      'Distance Base-Antenna 1 (cm)',
                      'Distance Base-Antenna 2 (cm)',
                      'Standard Deviation (cm)', save=False)

## GPS Data

In [ ]:
idx = -1
std_data = np.array([std_rpN, std_rpE, std_rpD, std_roll, std_yaw])

delaunay_surface_plot(distance_base_antenna1_test1, distance_base_antenna2_test1, std_data[idx][:distance_base_antenna1_test1.shape[0]], 
                      f'Relative error on {names[idx]} Component during experiment 1',
                      'Distance Base-Antenna 1 (cm)',
                      'Distance Base-Antenna 2 (cm)',
                      'Standard Deviation (cm)')

# Experiment 2

## Position Antenna 1

In [ ]:
# Experiment 2
delaunay_surface_plot(distance_antenna1_antenna2_test2, distance_base_antenna2_test2, delta_r[distance_base_antenna2_test1.shape[0]:],
                      'Relative error on the position of the antenna 1 during experiment 2',
                      'Distance Antenna 1 - Antenna 2 (cm)',
                      'Distance Base-Antenna 2 (cm)',
                      'Standard Deviation (cm)', save=False)

## GPS Data

In [ ]:
idx = -1
std_data = np.array([std_rpN, std_rpE, std_rpD, std_roll, std_yaw])

delaunay_surface_plot(distance_antenna1_antenna2_test2, distance_base_antenna2_test2, std_data[idx][distance_base_antenna1_test1.shape[0]:], 
                      f'Relative error on {names[idx]} Component during experiment 2',
                      'Distance Antenna 1 - Antenna 2 (cm)',
                      'Distance Base-Antenna 2 (cm)',
                      'Standard Deviation (cm)')

del idx, std_data

# Noise Power Spectrum

## Def useful functions

In [ ]:
timestep = (gps_antenna._datetime[1] - gps_antenna._datetime[0]).microseconds * 1e-6
print("Timestep : ", timestep, "s.")

def get_ps(array):
    """Function to compute the power spectrum of a given array.

    Parameters
    ----------
    array : array_like
        array containing the data to compute the power spectrum of.

    Returns
    -------
    power_spectrum : array_like
        array containing the power spectrum of the input array.
    freq: array_like
        array containing the frequency of the power spectrum.
    """
    N = array.size
    return np.abs(np.fft.rfft(array))**2, np.fft.rfftfreq(N, d=timestep/2)

In [ ]:
power_spectrum, freq = get_ps(gps_antenna.rpN[index_array[0]:index_array[int(index_array.size/2)]])
power_spectrum, freq = power_spectrum[1:], freq[1:]

plt.plot(freq, power_spectrum)
plt.xlabel('Frequency (Hz)')
plt.ylabel(r'Power Spectrum ($m^2/Hz$)')
plt.xscale('log')
plt.yscale('log')
plt.title('Noise Power Spectrum - North')

del power_spectrum, freq

## Save in Pickle file

In [ ]:
data_array = np.array([gps_antenna.rpN, gps_antenna.rpE, gps_antenna.rpD, gps_antenna.roll, gps_antenna.yaw])
data_names = np.array(["rpN", "rpE", "rpD", "roll", "yaw"])

dict_data = {}

for idata in range(len(data_names)):
    dict_data[data_names[idata]] = {}
    dict_data[data_names[idata]]["data"] = []
    dict_data[data_names[idata]]["ps"] = []
    dict_data[data_names[idata]]["freq"] = []
    
dict_data["datatime"] = gps_antenna._datetime

In [ ]:
for idata in range(5):
    for index in range(int(index_array.size/2)):
        dict_data[data_names[idata]]['data'].append(data_array[idata][index_array[index]:index_array[index+(int(index_array.size/2))]])
        ps, freq = get_ps(dict_data[data_names[idata]]['data'][-1])
        ps, freq = ps[1:], freq[1:]
        dict_data[data_names[idata]]['ps'].append(ps)
        dict_data[data_names[idata]]['freq'].append(freq)

In [ ]:
import pickle

with open('GPS_noise_analysis.pkl', 'wb') as f:
    data = pickle.dump(dict_data, f, protocol=pickle.HIGHEST_PROTOCOL)

## Fit 1/f noise with curve_fit

In [ ]:
def fit_ps(data, name, nbins=20, range=None, plot=False, return_=False, print_=False):
    power_spectrum, freq = get_ps(data)
    power_spectrum, freq = power_spectrum[1:], freq[1:]

    # Fit a line to the logarithm of the power spectrum
    res = linregress(np.log(freq), np.log(power_spectrum))
    slope, intercept, std_err = res.slope, res.intercept, res.stderr
    
    # Create bins for the power spectrum. Allows to understand the fitted line
    xx, yy, dx, dy, _ = ft.profile(np.log(freq), np.log(power_spectrum), nbins=nbins, plot=False, rng=range)
    res_bin = linregress(xx, yy)
    slope_bin, intercept_bin, std_err_bin = res_bin.slope, res_bin.intercept, res_bin.stderr

    if print_:
        print(f"Slope: {slope:.2f} " + "\u00B1" + f' {std_err:.2f}')

    if plot:
        # Plot the power spectrum and the fit
        plt.figure(figsize=(8, 6))
        plt.scatter(np.log(freq), np.log(power_spectrum), s=1, color='blue', alpha = 0.5, label='Power Spectrum')
        plt.errorbar(xx, yy, xerr=dx, yerr=dy, fmt='ro', label='Binning of the noise power spectrum')
        plt.plot(np.log(freq), slope * np.log(freq) + intercept, color='blue', label=f'Fit Power Spectrum - Slope: {slope:.2f} ' + "\u00B1" + f' {std_err:.2f}')
        
        plt.plot(np.log(freq), slope_bin * np.log(freq) + intercept_bin, color='red', label=f'Fit binned Power Spectrum - Slope: {slope_bin:.2f} ' + "\u00B1" + f' {std_err_bin:.2f}')


        plt.xlabel('Log(k)')
        plt.ylabel('Log(P(k))')
        plt.title('GPS Noise Power Spectrum - ' + name)
        plt.legend()
        plt.grid(True)
        plt.show()

    if return_:
        return slope_bin

### Fit on all configurations

In this section, I compute the power spectrum for all experimental configurations at same time, and then compute the associated slope for each GPS Data.

In [ ]:
gps_data = np.array([data_rpN, data_rpE, data_rpD, data_roll, data_yaw])

for idx, name in enumerate(names):
    fit_ps(gps_data[idx], name, nbins=20, plot=True)
    
del gps_data, idx, name

### Fit for each configurations

In this section, I compute the power spectrum for each experimental configurations, and then compute the mean slope for each GPS Data.

In [ ]:
def slope_each_config(data, name):
    slope_list = []
    for i in range(int(index_array.shape[0]/2)):
        data_array = data[index_array[i]:index_array[i+int(index_array.shape[0]/2)]]
        slope = fit_ps(data_array, name, nbins=20, return_=True)
        slope_list.append(slope)
    return np.array(slope_list)

In [ ]:
data, name = gps_antenna.yaw, "Yaw"

rpN_slopes = slope_each_config(data, name)
plt.plot(rpN_slopes)
plt.xlabel("Configuration Index")
plt.ylabel("Power Spectrum slope")
plt.title(f"Power Spectrum slope evolution - {name}")

del data, name, rpN_slopes, scatter

In [ ]:
data_gps = np.array([gps_antenna.rpN, gps_antenna.rpE, gps_antenna.rpD, gps_antenna.roll, gps_antenna.yaw])

for i,data in enumerate(data_gps):
    print(f'The mean slope for {names[i]} is : ', np.mean(slope_each_config(data, names[i])).round(2), "\u00B1", np.std(slope_each_config(data, names[i])).round(2))

## Fit more advanced noise model

$P_{noise}(f) = A_{white}^2 (1 + |f_{knee}/f|^{\alpha})$

In [ ]:
def noise_1_over_f(x, slope):
    return (1/x)**slope

def noise_model(x, A, f_knee, slope):
    return A**2 * (1 + np.abs(f_knee/x)**slope)

In [ ]:
def fit_noise_model(data, name, plot=False, _return=False, _print=False, binning = False, nbins=50):
    power_spectrum, freq = get_ps(data)
    power_spectrum, freq = power_spectrum[1:], freq[1:]

    if binning:
        freq, power_spectrum, dx, dy, _ = ft.profile(freq, power_spectrum, nbins=nbins, plot=False)

    params, err = curve_fit(noise_model, freq, power_spectrum, maxfev=10000)#, bounds=([0, 0, 0], [1e10, 1e10, 10]))

    if plot:
        plt.plot(freq, power_spectrum, label='Data')
        plt.plot(freq, noise_model(freq, *params), label='Fit')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Power Spectrum')
        plt.title(f'Noise Model Fit for {name}')
        plt.legend()
        plt.show()
    
    if _print:
        print(f"Noise model parameters for {name}: A = ", params[0].round(3), "\u00B1", err[0, 0].round(3), 
              "f_knee = ", params[1].round(3), "\u00B1", err[1, 1].round(3), 
              "slope = ", params[2].round(3), "\u00B1", err[2, 2].round(3))

    if _return:
        return params, err
    
def fit_noise_model_all(data, name, plot=False, _print=False, binning = False):
    params = []
    errors = []
    for i in range(int(index_array.size/2)):
        param, err = fit_noise_model(data[index_array[i]:index_array[i + int(index_array.size/2)]], name, plot, _return=True, _print=_print, binning = binning)
        params.append(param)
        errors.append(err)
    return np.array(params), np.array(errors) 

In [ ]:
params, errors = fit_noise_model_all(gps_antenna.rpN, 'North', _print=True, binning=False)

In [ ]:
fig, ax1 = plt.subplots(figsize = (15,5))
ax1plot = plt.errorbar(np.arange(0, 24), params[:, 0], yerr=errors[:, 0, 0], label=r"$A_{white}$", color='b')
ax2 = ax1.twinx()
ax2plot = plt.errorbar(np.arange(0, 24), params[:, 1], yerr=errors[:, 1, 1], label=r"$f_{knee}$", color='r')
ax3 = ax1.twinx()
ax3plot = plt.errorbar(np.arange(0, 24), params[:, 2], yerr=errors[:, 2, 2], label=r"$\alpha$", color='g')
ax3.spines["right"].set_position(("outward", 40))  
fig.legend(bbox_to_anchor=(1, 1), loc='upper left')
ax1.set_title('GPS Noise Model Parameters with Errors')
ax1.set_xlabel('Configuration Index')
ax1.set_ylabel('Parameter Value')
ax1.tick_params(axis="y", labelcolor="b")
ax2.tick_params(axis="y", labelcolor="r")
ax3.tick_params(axis="y", labelcolor="green")

In [ ]:
index = 2
# Triangle plot

n_samples = 10000
samples = np.random.multivariate_normal(mean=params[index], cov=errors[index], size=n_samples)

param_names = ["A_{white}", "f_{knee}", "alpha"]  
mcsamples = MCSamples(samples=samples, names=param_names, labels=param_names)

g = plots.get_subplot_plotter()
g.triangle_plot(mcsamples, filled=True, markers={r"$A_{white}$":params[index, 0], r"$f_{knee}$":params[index, 1], r"$\alpha$":params[index, 2]})
plt.suptitle(r"Fit of the noise model for a given configuration : $P_{noise}(f) = A_{white}^2 (1 + |\frac{f_{knee}}{f}|^{\alpha})$")

# Fit plot
plt.figure()
ps_test, freq_test = get_ps(gps_antenna.rpN[index_array[index]:index_array[index+(int(index_array.size/2))]])
freq_test, ps_test = freq_test[1:], ps_test[1:]
plt.plot(freq_test, ps_test, label="Noise Power Spectrum on North Position")
plt.errorbar(freq_test, noise_model(freq_test, params[index, 0], params[index, 1], params[index, 2]), 
             yerr=noise_model(freq_test, errors[index, 0, 0], errors[index, 1, 1], errors[index, 2, 2]), 
             fmt="o", label="Fitted Noise Power Spectrum")
plt.legend()
plt.xlabel('Frequency (Hz)')
plt.ylabel(r'Power Spectrum ($m^2/Hz$)')
plt.xscale("log")
plt.yscale("log")

In [ ]:
del ax1, ax1plot, ax2, ax2plot, ax3, ax3plot, data, errors, fig, freq_test, g, i, index, mcsamples, n_samples, param_names, params, ps_test, samples

## Fit with minuit

In [ ]:
def get_ps(array):
    """Function to compute the power spectrum of a given array.

    Parameters
    ----------
    array : array_like
        array containing the data to compute the power spectrum of.

    Returns
    -------
    power_spectrum : array_like
        array containing the power spectrum of the input array.
    freq: array_like
        array containing the frequency of the power spectrum.
    """
    N = array.size
    return np.abs(np.fft.rfft(array))**2, np.fft.rfftfreq(N, d=timestep/2)

def noise_model(x, A_white, f_knee, slope):
    return A_white**2 * (1 + np.abs(f_knee/x)**slope)

### Test Minuit

In [ ]:
index=6
data_name = 'rpN'
ps, freq= get_ps(gps_antenna.rpN[index_array[index]:index_array[index+(int(index_array.size/2))]])
ps, freq = ps[1:], freq[1:]

plt.plot(freq, ps)
plt.xlabel('Frequency (Hz)')
plt.ylabel(r'Power Spectrum ($m^2/Hz$)')
plt.xscale('log')
plt.yscale('log')
plt.title(f'Noise Power Spectrum - {data_name} - Configuration index = ' + str(index))
plt.show()

#### Test Linear binning

In [ ]:
binned_freq, binned_ps, binned_freq_error, binned_ps_error, _ = ft.profile(freq, ps, nbins=200, plot=True, log=False)
plt.plot(freq, ps)
plt.xlabel('Frequency (Hz)')
plt.ylabel(r'Power Spectrum ($m^2/Hz$)')
plt.xscale('log')
plt.yscale('log')
plt.title(f'Noise Power Spectrum - {data_name} - Configuration index = ' + str(index))
plt.show()

In [ ]:
least_squares = LeastSquares(binned_freq, binned_ps, binned_freq_error, noise_model)

m = Minuit(least_squares, A_white = 0.1, f_knee = 1, slope = 1)
m.limits["A_white"] = (0, None)
m.limits["f_knee"] = (0, None)
m.limits["slope"] = (0, None)
m.migrad()  # finds minimum of least_squares function
m.hesse()  # accurately computes uncertainties

In [ ]:
plt.plot(freq, ps, label="data")
plt.plot(freq, noise_model(freq, *m.values), 'r', label="fit")

# display legend with some fit info
fit_info = [
    f"$\\chi^2$/$n_\\mathrm{{dof}}$ = {m.fval:.1f} / {m.ndof:.0f} = {m.fmin.reduced_chi2:.1f}",
]
for p, v, e in zip(m.parameters, m.values, m.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

plt.legend(title="\n".join(fit_info), frameon=False)
plt.title('Fit on Noise Power Spectrum')
plt.xlabel('Frequency (Hz)')
plt.ylabel(r'Power Spectrum ($m^2/Hz$)')
plt.xscale("log")
plt.yscale("log")

In [ ]:
plt.errorbar(binned_freq, binned_ps, yerr=binned_ps_error, label="data")
plt.plot(binned_freq, noise_model(binned_freq, *m.values), 'r', label="fit")

# display legend with some fit info
fit_info = [
    f"$\\chi^2$/$n_\\mathrm{{dof}}$ = {m.fval:.1f} / {m.ndof:.0f} = {m.fmin.reduced_chi2:.1f}",
]
for p, v, e in zip(m.parameters, m.values, m.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

plt.legend(title="\n".join(fit_info), frameon=False)
plt.title("Fit on binned Noise Power Spectrum")
plt.xlabel('Frequency (Hz)')
plt.ylabel(r'Power Spectrum ($m^2/Hz$)')
plt.xscale("log")
plt.yscale("log")

#### Fit on the unbinned power spectrum

In [ ]:
ps_error = np.ones(ps.size) #* 0.1
least_squares = LeastSquares(freq, ps, ps_error, noise_model)

m = Minuit(least_squares, A_white = 0.1, f_knee = 1, slope = 1)
m.limits["A_white"] = (0, None)
m.limits["f_knee"] = (0, None)
m.limits["slope"] = (0, None)
m.migrad()  # finds minimum of least_squares function
m.hesse()  # accurately computes uncertainties

In [ ]:
plt.plot(freq, ps, label="data")
plt.plot(freq, noise_model(freq, *m.values), 'r', label="fit")

# display legend with some fit info
fit_info = [
    f"$\\chi^2$/$n_\\mathrm{{dof}}$ = {m.fval:.1f} / {m.ndof:.0f} = {m.fmin.reduced_chi2:.1f}",
]
for p, v, e in zip(m.parameters, m.values, m.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

plt.legend(title="\n".join(fit_info), frameon=False)
plt.title(f'Fit on Noise Power Spectrum - {data_name} - Instrumental Index {index}')
plt.xlabel('Frequency (Hz)')
plt.ylabel(r'Power Spectrum ($m^2/Hz$)')
plt.xscale("log")
plt.yscale("log")

#### Test Maximum Likelihood Estimation

In [ ]:
def nll_exp(A, f_knee, alpha, f, ps):
    P_model = noise_model(f, A, f_knee, alpha)
    return np.sum(np.log(P_model) + ps / P_model)

def nll_gauss(A, f_knee, alpha, freq, ps, sigma=binned_ps_error):
    P_model = noise_model(freq, A, f_knee, alpha)
    return np.sum(0.5 * ((ps - P_model) / sigma)**2 + 0.5 * np.log(2 * np.pi * sigma**2))

def nll_wrapper(A, f_knee, alpha):
    return nll_gauss(A, f_knee, alpha, binned_freq, binned_ps)

m = Minuit(nll_wrapper, A=0.1, f_knee=1.0, alpha=1.0)
m.limits['A'] = (0, None)
m.limits['f_knee'] = (0, None)
m.limits['alpha'] = (0, None)

m.migrad()
m.hesse()

In [ ]:
plt.plot(freq, ps, label="data")
plt.plot(freq, noise_model(freq, *m.values), 'r', label="fit")

# display legend with some fit info
fit_info = [
    f"$\\chi^2$/$n_\\mathrm{{dof}}$ = {m.fval:.1f} / {m.ndof:.0f} = {m.fmin.reduced_chi2:.1f}",
]
for p, v, e in zip(m.parameters, m.values, m.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

plt.legend(title="\n".join(fit_info), frameon=False)
plt.title(f'Fit on Noise Power Spectrum - {data_name} - Instrumental Index {index}')
plt.xlabel('Frequency (Hz)')
plt.ylabel(r'Power Spectrum ($m^2/Hz$)')
plt.xscale("log")
plt.yscale("log")

In [ ]:
del binned_freq, binned_freq_error, binned_ps, binned_ps_error, data_gps, e, fit_info, freq, least_squares, p, ps, ps_error, v

#### Test Code JC

In [ ]:
import qubic.scripts.users.DiversJC.CalibSalta.fitting as fit

In [ ]:
def white_1f(ff, args):
    white = args[0]
    fknee = args[1]
    power = args[2]
    values = np.zeros(len(ff))
    ok = ff !=0
    values[ok] = white**2 * (1. + (np.abs(ff[ok])/fknee)**power)
    return values

In [ ]:
signal = gps_antenna.rpN[index_array[index]:index_array[index+(int(index_array.size/2))]]

ps, freq = get_ps(signal)
ps, freq = ps[1:], freq[1:]

In [ ]:
errors = 1/np.sqrt(freq)
data = fit.Data(freq, ps, errors, white_1f)
guess =  np.array([np.std(signal), 0.1, -1])
m, ch2, ndf = data.fit_minuit(guess)

In [ ]:
params_name = [r"$A_{white}$", r"$f_{knee}$", r"alpha"]

# display legend with some fit info
fit_info = [
    f"$\\chi^2$/$n_\\mathrm{{dof}}$ = {m.fval:.1f} / {m.ndof:.0f} = {m.fmin.reduced_chi2:.1f}",
]
for p, v, e in zip(params_name, m.values, m.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

plt.plot(freq, ps, label = 'Data')
plt.plot(freq, white_1f(freq, data.fit), label = 'Fit')
plt.legend(title="\n".join(fit_info), frameon=False)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power Spectral Density (m$^2$/Hz)')
plt.xscale('log')
plt.yscale('log')
plt.title(f'Fit JC - {data_name} - Instrumental Index {index}')

for i in range(len(m.values)):
    print('{0} - Fitted : {1:5.3f} +/- {2:5.3f}'.format(params_name[i],m.values[i],m.errors[i]))

In [ ]:
stop

In [ ]:
del ch2, data, data_name, e, errors, fit_info, freq, guess, i, index, m, ndf, p, params_name, ps, signal, v

#### Same over all instrumental conf -> To test the stability over instrumental configurations

In [ ]:
data_name = 'rpN'
for index in range(int(index_array.size/2)):
    ps, freq = get_ps(gps_antenna.rpN[index_array[index]:index_array[index+(int(index_array.size/2))]])
    ps, freq = ps[1:], freq[1:]
    
    binned_freq, binned_ps, binned_freq_error, binned_ps_error, _ = ft.profile(freq, ps, nbins=250, plot=False, log=False)

    def nll_wrapper(A, f_knee, alpha):
        return nll_gauss(A, f_knee, alpha, binned_freq, binned_ps, sigma=binned_ps_error)

    m = Minuit(nll_wrapper, A=0.1, f_knee=1.0, alpha=1.0)
    m.limits['A'] = (0, None)
    m.limits['f_knee'] = (0, None)
    m.limits['alpha'] = (0, None)

    m.migrad()
    m.hesse()
    
    plt.plot(freq, ps, label="data")
    plt.plot(freq, noise_model(freq, *m.values), 'r', label="fit")

    # display legend with some fit info
    fit_info = [
        f"$\\chi^2$/$n_\\mathrm{{dof}}$ = {m.fval:.1f} / {m.ndof:.0f} = {m.fmin.reduced_chi2:.1f}",
    ]
    for p, v, e in zip(m.parameters, m.values, m.errors):
        fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

    plt.legend(title="\n".join(fit_info), frameon=False)
    plt.title(f'Fit on Noise Power Spectrum - {data_name} - Instrumental Index {index}')
    plt.xlabel("x")
    plt.ylabel("y")
    plt.xscale("log")
    plt.yscale("log")
    plt.show()
        
    del ps, freq, binned_freq, binned_ps, binned_freq_error, binned_ps_error, m, nll_wrapper, fit_info

### Fit using LeastSquares with minuit

In [ ]:
def fit_ps_minuit(data, model, nbins=200, plot=False, data_name=None, index=None):
    ps, freq = get_ps(data)
    ps, freq = ps[1:], freq[1:]
    
    x, y, _, dy, _ = ft.profile(freq, ps, nbins=nbins, plot=False)
    
    least_squares = LeastSquares(x, y, dy, model)
    m = Minuit(least_squares, A_white = 1, f_knee = 1, slope = 1)
    m.limits["A_white"] = (0, None)
    m.limits["f_knee"] = (0, None)
    m.limits["slope"] = (0, None)
    m.migrad()  # finds minimum of least_squares function
    m.hesse()  # accurately computes uncertainties
    
    if plot:
        plt.plot(freq, ps, label="data")
        plt.plot(freq, noise_model(freq, *m.values), 'r', label="fit")

        # display legend with some fit info
        fit_info = [
            f"$\\chi^2$/$n_\\mathrm{{dof}}$ = {m.fval:.1f} / {m.ndof:.0f} = {m.fmin.reduced_chi2:.1f}",
        ]
        for p, v, e in zip(m.parameters, m.values, m.errors):
            fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")
            
        plt.title(f'Fit on Noise Power Spectrum - {data_name} - Instrumental Index {index}')
        plt.legend(title="\n".join(fit_info), frameon=False)
        plt.xlabel('Frequency (Hz)')
        plt.ylabel(r'Power Spectrum ($m^2/Hz$)')
        plt.xscale("log")
        plt.yscale("log")
        plt.show()
    
    return m.values, m.errors, m.fmin.reduced_chi2

In [ ]:
params_values_least_squares, params_errors_least_squares, reduced_chi2_least_squares = [], [], []
data = [gps_antenna.rpN, gps_antenna.rpE, gps_antenna.rpD, gps_antenna.roll, gps_antenna.yaw]
label = [r'$A_{white}$', r'$f_{knee}$', r'$\alpha$']

# Fit for each instrumental configurations
for idata in range(len(data)):
    values, errors, chi2_dof= [], [], []
    for index in range(int(index_array.size/2)):
        val, err, red_chi2 = fit_ps_minuit(data[idata][index_array[index]:index_array[index+(int(index_array.size/2))]], noise_model, nbins=300, plot=False, data_name=names[idata], index=index)
        values.append(val)
        errors.append(err)
        chi2_dof.append(red_chi2)
    params_values_least_squares.append(values)
    params_errors_least_squares.append(errors)
    reduced_chi2_least_squares.append(chi2_dof)
    
print(len(params_values_least_squares))
print(len(params_values_least_squares[0]))
print(len(params_values_least_squares[0][0]))

In [ ]:
# IMPORTANT: the mean is not computed in a proper way, as I don't ponderate the different number of points in each dataset

plt.style.use('seaborn-v0_8-whitegrid')
fig, ax = plt.subplots(3, 5, figsize=(15, 10), sharey='row', sharex=True)

n_datasets = len(data)
n_points   = len(params_values_least_squares[0])  
for idata in range(n_datasets):
    for iparam in range(3):
        values = [params_values_least_squares[idata][i][iparam] for i in range(n_points)]
        errors = [params_errors_least_squares[idata][i][iparam] for i in range(n_points)]
        x = np.arange(n_points)
        
        ax[iparam, idata].errorbar(x, values, yerr=errors,
                                   fmt='o', color='black', capsize=3)
        
        mean_val = np.mean(values)
        std_val = np.std(values)
        ax[iparam, idata].axhline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Mean = {mean_val:.2f} | Std = {std_val:.2f}')
        ax[iparam, idata].legend(fontsize=8)

for iparam in range(3):
    ax[iparam, 0].set_ylabel(label[iparam], fontsize=12)

for idata in range(n_datasets):
    ax[0, idata].set_title(names[idata], fontsize=14)
#    ax[1, idata].set_ylim(0, 10)

for idata in range(n_datasets):
    ax[2, idata].set_xlabel('Instrumental Index', fontsize=12)
fig.suptitle(r'GPS noise analysis - LeastSquares - $P_{noise}(f) = A_{white}^2 (1 + |f_{knee}/f|^{\alpha})$', fontsize=16)
plt.tight_layout()
plt.savefig('GPS noise analysis - LeastSquares.png')
plt.show()

In [ ]:
plt.style.use('seaborn-v0_8-whitegrid')
fig, ax = plt.subplots(1, 5, figsize=(15, 5), sharey='row', sharex=True)

n_datasets = len(data)
n_points   = len(params_values_least_squares[0])  
for idata in range(n_datasets):
    red_chi2 = [reduced_chi2_least_squares[idata][i] for i in range(n_points)]
    x = np.arange(n_points)
    
    ax[idata].scatter(x, red_chi2,
                                marker='x', color='black')
    
    mean_red_chi2 = np.mean(red_chi2)
    std_red_chi2 = np.std(red_chi2)
    ax[idata].axhline(mean_red_chi2, color='red', linestyle='--', linewidth=2, label=f'Mean = {mean_red_chi2:.2f} | Std = {std_red_chi2:.2f}')
    ax[idata].legend(fontsize=8)
    
for idata in range(n_datasets):
    ax[idata].set_title(names[idata], fontsize=14)
    ax[idata].set_xlabel('Instrumental Index', fontsize=12)
    
ax[0].set_ylabel(r'$\chi^2 / n_{dof}$')

fig.suptitle(r'GPS noise analysis - $P_{noise}(f) = A_{white}^2 (1 + |f_{knee}/f|^{\alpha})$', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
del ax, chi2_dof, data, e, err, errors, fig, idata, index, iparam, label, mean_red_chi2, mean_val, n_datasets, n_points, p, red_chi2, reduced_chi2_least_squares, std_red_chi2, std_val, val, values, x

### Fit using my Gaussian LogLikelihood function with minuit

In [ ]:
def nll_exp(A, f_knee, alpha, f, ps):
    P_model = noise_model(f, A, f_knee, alpha)
    return np.sum(np.log(P_model) + ps / P_model)

def nll_gauss(A, f_knee, alpha, freq, ps, sigma):
    P_model = noise_model(freq, A, f_knee, alpha)
    return np.sum(0.5 * ((ps - P_model) / sigma)**2 + 0.5 * np.log(2 * np.pi * sigma**2))

def fit_minuit_ll(data, nbins=300, plot=False, data_name=None, index=None):
    ps, freq = get_ps(data)
    ps, freq = ps[1:], freq[1:]
    
    binned_freq, binned_ps, _, binned_ps_error, _ = ft.profile(freq, ps, nbins=nbins, plot=False)
    
    def nll_wrapper(A, f_knee, alpha):
       return nll_gauss(A, f_knee, alpha, binned_freq, binned_ps, sigma=binned_ps_error)

    m = Minuit(nll_wrapper, A=0.1, f_knee=1, alpha=1)
    m.limits['A'] = (0, None)
    m.limits['f_knee'] = (0, None)
    m.limits['alpha'] = (0, None)

    m.migrad()
    m.hesse()
    
    if plot:
        plt.plot(freq, ps, label="data")
        plt.plot(freq, noise_model(freq, *m.values), 'r', label="fit")

        # display legend with some fit info
        fit_info = [
            f"$\\chi^2$/$n_\\mathrm{{dof}}$ = {m.fval:.1f} / {m.ndof:.0f} = {m.fmin.reduced_chi2:.1f}",
        ]
        for p, v, e in zip(m.parameters, m.values, m.errors):
            fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")
        plt.title(f'Fit on Noise Power Spectrum - {data_name} - Instrumental Index {index}')
        plt.legend(title="\n".join(fit_info), frameon=False)
        plt.xlabel('Frequency (Hz)')
        plt.ylabel(r'Power Spectrum ($m^2/Hz$)')
        plt.xscale("log")
        plt.yscale("log")
        plt.show()
        
    return m.values, m.errors, m.fmin.reduced_chi2

In [ ]:
params_values_loglike, params_errors_loglike, reduced_chi2_loglike = [], [], []
data = [gps_antenna.rpN, gps_antenna.rpE, gps_antenna.rpD, gps_antenna.roll, gps_antenna.yaw]
names = ['rpN', 'rpE', 'rpD', 'roll', 'yaw']
label = [r'$A_{white}$', r'$f_{knee}$', r'$\alpha$']

# Fit for each instrumental configurations
for idata in range(len(data)):
    values, errors, chi2_dof= [], [], []
    for index in range(int(index_array.size/2)):
        val, err, red_chi2 = fit_minuit_ll(data[idata][index_array[index]:index_array[index+(int(index_array.size/2))]], nbins=300, plot=False, data_name=names[idata], index=index)
        values.append(val)
        errors.append(err)
        chi2_dof.append(red_chi2)
    params_values_loglike.append(values)
    params_errors_loglike.append(errors)
    reduced_chi2_loglike.append(chi2_dof)
    
print(len(params_values_loglike))
print(len(params_values_loglike[0]))
print(len(params_values_loglike[0][0]))

In [ ]:
plt.style.use('seaborn-v0_8-whitegrid')
fig, ax = plt.subplots(3, 5, figsize=(15, 10), sharey='row', sharex=True)

n_datasets = len(data)
n_points   = len(params_values_loglike[0])  
for idata in range(n_datasets):
    for iparam in range(3):
        values = [params_values_loglike[idata][i][iparam] for i in range(n_points)]
        errors = [params_errors_loglike[idata][i][iparam] for i in range(n_points)]
        x = np.arange(n_points)
        
        ax[iparam, idata].errorbar(x, values, yerr=errors,
                                   fmt='o', color='black', capsize=3)
        
        mean_val = np.mean(values)
        std_val = np.std(values)
        ax[iparam, idata].axhline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Mean = {mean_val:.2f} | Std = {std_val:.2f}')
        ax[iparam, idata].legend(fontsize=8)

for iparam in range(3):
    ax[iparam, 0].set_ylabel(label[iparam], fontsize=12)

for idata in range(n_datasets):
    ax[0, idata].set_title(names[idata], fontsize=14)
#    ax[1, idata].set_ylim(0, 10)

for idata in range(n_datasets):
    ax[2, idata].set_xlabel('Instrumental Index', fontsize=12)
fig.suptitle(r'GPS noise analysis - logLikelihood - $P_{noise}(f) = A_{white}^2 (1 + |f_{knee}/f|^{\alpha})$', fontsize=16)
plt.tight_layout()
plt.savefig('GPS noise analysis - logLikelihood.png')
plt.show()

In [ ]:
del ax, chi2_dof, data, data_name, err, errors, fig, idata, index, iparam, label , mean_val, n_datasets, n_points, red_chi2, reduced_chi2_loglike, std_val, v, val, values, x

### Are the parameters scaled with the distance between antennas ?

In [ ]:
params_value_north = np.array(params_values_loglike, dtype=float)[0]
print(params_value_north.shape)

### Noise parameters vs distance

In [ ]:
print("Distance between antennas : ", distance_antenna1_antenna2_test2)
print("Distance between base and antenna 2 : " , distance_base_antenna2_test2)

In [ ]:
discrete_values = [100, 200, 300]
colors = ['#EE6677', '#228833', '#4477AA']  

# Create a 1-row, 2-column subplot figure
fig = make_subplots(
    rows=1, 
    cols=2, 
    # subplot_titles=[
    #     "Noise of the North component data: \(f_{knee}\)",
    #     "Noise of the North component data: \(A_{white}\)"
    # ]
)

# ----------- First subplot: f_{knee} -----------
for val, col in zip(discrete_values, colors):
    mask = (distance_base_antenna2_test2 == val)
    
    x_vals = distance_antenna1_antenna2_test2[mask]
    # Here we take the column for f_{knee} (index 1)
    y_vals = params_value_north[int(distance_base_antenna1_test1.shape[0]):, 1][mask]
    
    fig.add_trace(
        go.Scatter(
            x=x_vals,
            y=y_vals,
            mode='markers',
            marker=dict(size=10, color=col),
            name=f"{val} cm",
                        showlegend=False  # This removes the legend for the second subplot

        ),
        row=1,
        col=1
    )

# ----------- Second subplot: A_{white} -----------
for val, col in zip(discrete_values, colors):
    mask = (distance_base_antenna2_test2 == val)
    
    x_vals = distance_antenna1_antenna2_test2[mask]
    # Here we take the column for A_{white} (index 0)
    y_vals = params_value_north[int(distance_base_antenna1_test1.shape[0]):, 0][mask]
    
    fig.add_trace(
        go.Scatter(
            x=x_vals,
            y=y_vals,
            mode='markers',
            marker=dict(size=10, color=col),
            name=f"{val} cm"
        ),
        row=1,
        col=2
    )

# Update axis labels for each subplot
fig.update_xaxes(title='Distance between antennas (cm)', row=1, col=1, gridcolor='lightgray')
fig.update_xaxes(title='Distance between antennas (cm)', row=1, col=2, gridcolor='lightgray')
fig.update_yaxes(title=r'$f_{knee}$', row=1, col=1, gridcolor='lightgray')
fig.update_yaxes(title=r'$A_{white}$', row=1, col=2, gridcolor='lightgray')

# Update overall layout
fig.update_layout(
    title='Noise parameters of the North component data',
    legend_title='Distance between base and antenna 2',
    width=1500,
    height=900,
    template='plotly_white',   # sets a clean background theme
    paper_bgcolor='white',     # ensures the outer area is white
    plot_bgcolor='white'       # ensures the plotting area is white
)

fig.show()

#### Least Squares

In [ ]:
# Experiment 1
delaunay_surface_plot(distance_base_antenna1_test1, distance_base_antenna2_test1, params_value_north[:distance_base_antenna1_test1.shape[0], 0], 
                      'A_white during experiment 1 for North Component',
                      'Distance Base-Antenna 1 (cm)',
                      'Distance Base-Antenna 2 (cm)',
                      'A_white', save=False)

In [ ]:
# Experiment 1
delaunay_surface_plot(distance_base_antenna1_test1, distance_base_antenna2_test1, params_value_north[:distance_base_antenna1_test1.shape[0], 1], 
                      'f_knee during experiment 1 for North Component',
                      'Distance Base-Antenna 1 (cm)',
                      'Distance Base-Antenna 2 (cm)',
                      'f_knee', save=False)

In [ ]:
# Experiment 2
delaunay_surface_plot(distance_antenna1_antenna2_test2, distance_base_antenna2_test2, params_value_north[distance_base_antenna1_test1.shape[0]:, 0], 
                      'A_white during experiment 2 for North Component',
                      'Distance Antenna 1-Antenna 2 (cm)',
                      'Distance Base-Antenna 2 (cm)',
                      'A_white', save=False)

In [ ]:
# Experiment 2
delaunay_surface_plot(distance_antenna1_antenna2_test2, distance_base_antenna2_test2, params_value_north[distance_base_antenna1_test1.shape[0]:, 1], 
                      'f_knee during experiment 2 for North Component',
                      'Distance Antenna 1-Antenna 2 (cm)',
                      'Distance Base-Antenna 2 (cm)',
                      'f_knee', save=False)

#### Log Likelihood

In [ ]:
# Experiment 1
delaunay_surface_plot(distance_base_antenna1_test1, distance_base_antenna2_test1, params_value_north[:distance_base_antenna1_test1.shape[0], 0], 
                      'A_white during experiment 1 for North Component',
                      'Distance Base-Antenna 1 (cm)',
                      'Distance Base-Antenna 2 (cm)',
                      'A_white', save=False)

In [ ]:
# Experiment 1
delaunay_surface_plot(distance_base_antenna1_test1, distance_base_antenna2_test1, params_value_north[:distance_base_antenna1_test1.shape[0], 1], 
                      'f_knee during experiment 1 for North Component',
                      'Distance Base-Antenna 1 (cm)',
                      'Distance Base-Antenna 2 (cm)',
                      'f_knee', save=False)

In [ ]:
# Experiment 2
delaunay_surface_plot(distance_antenna1_antenna2_test2, distance_base_antenna2_test2, params_value_north[distance_base_antenna1_test1.shape[0]:, 0], 
                      'A_white during experiment 2 for North Component',
                      'Distance Antenna 1-Antenna 2 (cm)',
                      'Distance Base-Antenna 2 (cm)',
                      'A_white', save=False)

In [ ]:
# Experiment 2
delaunay_surface_plot(distance_antenna1_antenna2_test2, distance_base_antenna2_test2, params_value_north[distance_base_antenna1_test1.shape[0]:, 1], 
                      'f_knee during experiment 2 for North Component',
                      'Distance Antenna 1-Antenna 2 (cm)',
                      'Distance Base-Antenna 2 (cm)',
                      'f_knee', save=False)